<a href="https://colab.research.google.com/github/meghutch/Breast-Cancer-Classification-Clinical-Genomic/blob/master/PCA_TroubleShooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Clinical Outcomes of Breast Cancer Patients - Gene Expression Data**

## **Principal Component Analysis**

**Author:** Meg Hutch

**Date:** November 15, 2019

**Objective:** Apply Principal Component Analysis to Determine Which Genes to Include into our Neural Network 

**This specific version troubleshoots some discrepancies I found while reasearching these methods!**

****Update: I did some testing and these methods don't seem to be causing significant differences. I will probably just need to later validate whether the PC change using the different methods, but for now, I think I can go ahead, clean up the code****

**References**

* https://www.youtube.com/watch?v=Lsue2gEM9D0&list=PLblh5JKOoLUIcdlgu78MnlATeyx4cEVeR&index=54&t=0s

* https://www.youtube.com/watch?v=FgakZw6K1QQ

* https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60 (**concerned that perhaps error in the code regarding the fit_transform function**)

* https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python 

* https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c

In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [0]:
# Connect Colab to google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Import Data
gene_data = pd.read_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Data/merged_expression.txt', sep=',')

In [0]:
#gene_data.head()
#genes.info()

# **Data Pre-Processing**

**Check if NAs**

In [0]:
gene_data.isna().any()

**Remove cases where there are missing values**

In [0]:
gene_data = gene_data.dropna()

**Check number of patients after removing Nas**

In [0]:
gene_data.info()

**Create a dataset just containing gene expression**

In [0]:
# remove event, os_motnhs, and Five_Year from the dataset
genes = gene_data.drop(columns=["Unnamed: 0", "EVENT", "OS_MONTHS", "FIVE_YEAR"])

# **Principal Component Analysis**

**What is a Pricipal Component? (From DataCamp)**

Principal componenets have both direction and magnitude. The direction represents across which principal axes the data is mostly spread out or has the most variance and the magnitude signifies the amount of varaince that Principal Component captures of the data when projected onto that axis. 

The principal components are a straight line, and the first principal component holds the most variance in the data. Each subsequent prinicpal component is orthogonal to the last and has a lesser variance .

Correlated features contribute to the same principal component, thereby reducing the original data features into uncorrelated prinicpal components; each representing a different set of correlated features with differents of variation.

Each principal component represents a percentage of total variation captured from the data

****Note:** I troubleshooted some discrepancies among the references above in regards to fit and fit transform. Some articles would initially use fit_transform to scale and then again used pca.fit_transform when identifying prinicipal components, other references, instead of fit_transform just used pca.transform for this step. I tested both methods and got the same results. Actually looks like maybe using fit_transform is probably just comnbining these steps?****** 


**1) Standardize the Data**

Must scale features in your data before applying PCA. **StandardScaler** helps standardize features onto unit scale (mean = 0 and standard deviation = 1). Thus, each value in the dataset will have the sample mean value subtracted and then divided by the standard deviation of the whole dataset. 


In [0]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

# Stanardize/Scale the data
x = StandardScaler().fit_transform(genes)

**Let's check whether the normalized data has a mean of zero and a standard deviation of 1**

In [51]:
np.mean(x), np.std(x)

(4.1760406566772064e-19, 0.9999999999999999)

**Troubleshoot:**

I was seeing differences in some of my references regarding whether the pca function should be followed by fit(x) or fit_transform(x) where x is our scaled data. 

As an update, I tried both methods, 

In [0]:
#11.19.2019 EDIT/TEST - this is like the youtube video https://www.youtube.com/watch?v=Lsue2gEM9D0&list=PLblh5JKOoLUIcdlgu78MnlATeyx4cEVeR&index=54&t=0s 
pca = PCA()
pca.fit(x)
test1 = pca.transform(x)
test1 = pd.DataFrame(test1)
test1

In [0]:
# test 2 - fit_transform is used twice, does this have any repercussions?
# From  https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
# Data camp only looks to follow this method - https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python
pca = PCA()
test2 = pca.fit_transform(x)
test2 = pd.DataFrame(test2)
test2

In [0]:
# determine whether the data frames are the same
test1.equals(test2) # this indicates that they are not

In [0]:
#why is this false?
test3 = pd.concat([test1,test2]).drop_duplicates(keep=False)

In [0]:
test2[ ~test2.isin(test1)].dropna()

In [0]:
test3 = test1 != test2

test3[0].value_counts()

In [0]:
test3.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Data/pca_test.csv')
test1.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Data/pca_test1.csv')
test2.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Data/pca_test2.csv')

I manually evaluated the discrepancies by exporting each test and the true/false analysis to excel. 

From my trouble shooting, it looks like discrepancies are like 10 decimal places in. I'm not sure why this is occcuring, maybe a rounding issue, but I don't think this is a huge problem to stress about right now - because most of the sources seem to be using the fit_transform , I will probably start with that. I can then always validate and see whether the Prinicipal components differ. I don't believe we use this information in the neural network - we would just use the genes 